<a href="https://colab.research.google.com/github/Jonathangadeaharder/GameTheoryMA/blob/Testing2/True_Experimente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
import copy
def mutate(s, m):
    """Flips each bit of s with a probability of 1/m."""
    return [bit if random.random() > 1/m else 1-bit for bit in s]

def calculate_hamming_distance(sol1, sol2):
    """Calculate the Hamming distance between two solutions."""
    ham = sum(el1 != el2 for el1, el2 in zip(sol1, sol2))
    return ham

def calculate_diversity(P):
    """
    Calculates the diversity of the population, considering only unique solutions.

    Parameters:
    P (list): The population, each solution is a list of indices.

    Returns:
    int: The total pairwise Hamming distance among all unique solutions.
    """
    # Remove duplicates and consider only unique solutions
    unique_solutions = set(tuple(sol) for sol in P)
    unique_solutions_list = list(unique_solutions)  # Convert back to list for indexing

    diversity = 0
    for i in range(len(unique_solutions_list)):
        for j in range(i + 1, len(unique_solutions_list)):
            diversity += calculate_hamming_distance(unique_solutions_list[i], unique_solutions_list[j])

    return diversity

def is_valid_maximum_matching(s, l, r):
    """
    Check if the solution 's' represents a valid maximum matching in a bipartite graph.

    Parameters:
    s (list): Solution represented as a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.

    Returns:
    bool: True if the solution represents a valid maximum matching, False otherwise.
    """
    matched_right = set()
    for i in range(r):
        matched_left = -1
        for j in range(l):
            if s[i * l + j] == 1:
                if matched_left != -1:  # More than one match for a right vertex
                    return False
                matched_left = j
        if matched_left != -1:
            if matched_left in matched_right:  # Duplicate match found
                return False
            matched_right.add(matched_left)

    # Check if all right vertices are matched (for maximum matching)
    return len(matched_right) == r


def generate_initial_population(mu, l, r):
    """
    Generates an initial population of solutions where each r_i is mapped to l_i for i=0 to i=r.
    """
    m = l * r
    initial_population = []
    for _ in range(mu):
        solution = [0] * m
        for i in range(min(l, r)):  # To handle cases where l != r
            index = i * l + i  # Mapping r_i to l_i
            solution[index] = 1
        initial_population.append(solution)
    return initial_population


def print_solution_as_indices(s, l, r):
    """
    Prints a single solution as an array of indices showing the matchings.

    Parameters:
    s (list): The solution to print, represented as a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    """
    for i in range(r):
        matched_index = -1  # Default value if no match is found
        for j in range(l):
            if s[i * l + j] == 1:
                matched_index = j
                break
        print(matched_index, end=" ")
    print()  # New line at the end of the solution

def print_population_as_matrix(P, l, r):
    """
    Prints the entire population as a matrix, each row representing a solution.

    Parameters:
    P (list): The population to print, each solution is a binary string of length r*l.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    """
    for solution in P:
        print_solution_as_indices(solution, l, r)

def ead_algorithm_for_matching(mu, l, r, log=False):
    """
    Implementation of the EAD algorithm adapted for a bipartite graph matching problem.

    Parameters:
    mu (int): Population size.
    l (int): Number of vertices on the left side of the bipartite graph.
    r (int): Number of vertices on the right side of the bipartite graph.
    log (bool): If set to True, logs the progress of the algorithm.

    Returns:
    tuple: A tuple containing the final population and the number of fitness evaluations.
    """
    P = generate_initial_population(mu, l, r)
    fitness_evaluations = 0
    m = l * r
    steps=0
    while calculate_diversity(P) < mu * r * (mu-1):
        s = random.choice(P)
        s_prime = mutate(s, m)
        steps+=1
        if is_valid_maximum_matching(s_prime, l, r) and s != s_prime:
            P.append(s_prime)
            fitness_evaluations += 1
            # Calculate and print contributions for each solution including s_prime
            contributions = []
            total_div = calculate_diversity(P)
            for i in range(len(P)):
                tmp = P[i]
                P.pop(i)
                div = calculate_diversity(P)
                P.insert(i,tmp)
                contr = total_div-div
                contributions.append((P[i],contr))
            if log:
                print("\nContributions:")
                for sol, contrib in contributions:
                    print_solution_as_indices(sol, l, r)
                    print(f": {contrib}")
                print("\n")

            # Find and remove the solution with the minimum contribution
            min_contribution = min(contributions, key=lambda x: x[1])[1]
            min_contributors = [sol for sol, contrib in contributions if contrib == min_contribution]
            z = random.choice(min_contributors)
            P.remove(z)
            # Print current diversity and fitness evaluations
            if log:
                print(f"Current Diversity: {calculate_diversity(P)}, Fitness Evaluations: {fitness_evaluations}")

    return steps, fitness_evaluations

In [3]:

import random
import numpy as np
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def run_experiment(l, r, mu):
    steps, fitness_evaluations = ead_algorithm_for_matching(mu, l, r, log=False)
    return (steps,fitness_evaluations)

# Experiment parameters
l = 30  # Number of vertices on the left side of the bipartite graph
r = 20  # Number of vertices on the right side of the bipartite graph
mu_values = range(2, 10, 1)

# File to store experiment results
log_file = "ead_algorithm_runs.txt"

# Run the experiments and log results
with open(log_file, 'w') as file:
    with ProcessPoolExecutor() as executor:
        file.write("EAD Algorithm Experiment Runs\n")
        file.write("l = {}, r = {}\n".format(l, r))
        file.write("μ Values: {}\n\n".format(list(mu_values)))

        for mu in tqdm(mu_values):
            # Execute multiple runs in parallel
            futures = [executor.submit(run_experiment, l, r, mu) for _ in range(3)]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
            fitness_evaluations_list = [x[1] for x in results]
            steps_list = [x[0] for x in results]

            file.write("μ = {}: {}\n".format(mu, fitness_evaluations_list))
            average = np.mean(fitness_evaluations_list)
            std_dev = np.std(fitness_evaluations_list)
            file.write(f"Average Fitness Evaluations: {average}, Standard Deviation: {std_dev}\n\n")

            file.write("μ = {}: {}\n".format(mu, steps_list))
            average = np.mean(steps_list)
            std_dev = np.std(steps_list)
            file.write(f"Average Steps: {average}, Standard Deviation: {std_dev}\n\n")
print("Experiments completed. Results logged in", log_file)

 12%|█▎        | 1/8 [16:40<1:56:40, 1000.00s/it]


KeyboardInterrupt: 

In [ ]:
import random
import numpy as np
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

def run_experiment(l, r, mu=10):
    # Assuming ead_algorithm_for_matching is defined and returns steps, fitness_evaluations
    steps, fitness_evaluations = ead_algorithm_for_matching(mu, l, r, log=False)
    return (steps, fitness_evaluations)

# Fixed mu value
mu = 10

# File to store experiment results
log_file = "ead_algorithm_runtime_analysis.txt"

# Run the experiments and log results
with open(log_file, 'w') as file:
    with ProcessPoolExecutor() as executor:
        file.write("EAD Algorithm Runtime Analysis\n")
        file.write("Fixed μ = {}\n\n".format(mu))

        # Iterate over the range of l and r values
        for l in tqdm(range(32, 43)):  # Adjust the range as needed
            r = l - 11
            file.write(f"l = {l}, r = {r}\n")

            # Execute multiple runs in parallel
            futures = [executor.submit(run_experiment, l, r) for _ in range(3)]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]

            fitness_evaluations_list = [x[1] for x in results]
            steps_list = [x[0] for x in results]

            file.write("Fitness Evaluations: {}\n".format(fitness_evaluations_list))
            average_fitness = np.mean(fitness_evaluations_list)
            std_dev_fitness = np.std(fitness_evaluations_list)
            file.write(f"Average Fitness Evaluations: {average_fitness}, Standard Deviation: {std_dev_fitness}\n")

            file.write("Steps: {}\n".format(steps_list))
            average_steps = np.mean(steps_list)
            std_dev_steps = np.std(steps_list)
            file.write(f"Average Steps: {average_steps}, Standard Deviation: {std_dev_steps}\n\n")

print("Runtime analysis completed. Results logged in", log_file)


  0%|          | 0/11 [00:00<?, ?it/s]